<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/Cosinor_phase_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# データの読み込み
csv_file_path = '/content/2024-0923-1048_Plate1_F0_N_D.csv'

# メタデータの10行をスキップして、正しいラベルを含むデータを読み込み
data = pd.read_csv(csv_file_path, skiprows=11)

# コサイン関数の定義 (周期 T が既知)
T = 24  # 周期を24時間と仮定

def cosinor_model(t, mesor, amplitude, phase):
    """
    t: 時間
    mesor: 平均レベル (MESOR)
    amplitude: 振幅
    phase: acrophase (位相)
    """
    return mesor + amplitude * np.cos(2 * np.pi * t / T - phase)

# 各測定項目の「Time(h)」と「RLU」をペアで処理
measurement_pairs = []
for i in range(1, len(data.columns), 2):  # Time(h) と RLU をペアにする
    time_col = data.columns[i]  # "Time(h)"
    rlu_col = data.columns[i+1]  # "RLU"
    measurement_pairs.append((time_col, rlu_col))

# 特定の時間帯を設定（例：時間の開始と終了）
start_time = 7.0  # 解析したい時間範囲の開始（単位: 時間）
end_time = 30.0   # 解析したい時間範囲の終了（単位: 時間）

# コサイナー解析の結果を保存するリスト
results = []

# 各測定項目に対してコサイナー解析を実行
for time_col, rlu_col in measurement_pairs:
    # 各測定項目の時間データと RLU データを取得し、数値に変換
    time_data = pd.to_numeric(data[time_col], errors='coerce')
    rlu_data = pd.to_numeric(data[rlu_col], errors='coerce')

    # 特定の時間範囲にデータをフィルタリング
    valid_indices = (time_data >= start_time) & (time_data <= end_time) & (rlu_data.notna())
    time_data_filtered = time_data[valid_indices]
    rlu_data_filtered = rlu_data[valid_indices]

    # フィルタされたデータが空でないことを確認
    if len(time_data_filtered) > 0 and len(rlu_data_filtered) > 0:
        # 初期値を設定してコサイン関数にフィット
        initial_guess = [np.mean(rlu_data_filtered), (np.max(rlu_data_filtered) - np.min(rlu_data_filtered)) / 2, 0.0]
        params, params_covariance = curve_fit(cosinor_model, time_data_filtered, rlu_data_filtered, p0=initial_guess)

        # フィット結果の取得
        fitted_mesor, fitted_amplitude, fitted_phase = params

        # 結果を保存
        results.append({
            'Measurement': rlu_col,
            'MESOR': fitted_mesor,
            'Amplitude': fitted_amplitude,
            'Phase (radians)': fitted_phase
        })

# コサイナー解析の結果をデータフレームとして表示
results_df = pd.DataFrame(results)

# 振幅を絶対値に変換
results_df['Amplitude'] = results_df['Amplitude'].abs()

# 解析結果をCSVファイルに保存
output_csv_file_path = '/content/cosinor_analysis_results.csv'
results_df.to_csv(output_csv_file_path, index=False)

print(f"Results with absolute amplitude saved to {output_csv_file_path}")

# 結果のプレビュー表示
print(results_df.head())

Results with absolute amplitude saved to /content/cosinor_analysis_results.csv
  Measurement        MESOR    Amplitude  Phase (radians)
0         RLU  1183.970373    83.079112         0.044558
1       RLU.1    37.340763  1791.987622         0.824111
2       RLU.2    94.677396  2081.350946         0.838419
3       RLU.3    89.359059  2098.833220         0.851322
4       RLU.4   122.712252  2263.912852         0.981910
